# Resnet

> A 1D Resnet

In [ ]:
#| default_exp model.resnet

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev.cli import *

In [ ]:
#| export
from fastai.vision.all import *
from fastai.data.all import *
from pathlib import Path
import pandas as pd

from spanda.data.load import (get_spectra_files, 
                              SpectraBlock,
                              get_target,
                              random_weighted_avg)
                              
from fastai.layers import ConvLayer
from fastai.vision.models import xresnet

In [ ]:
tst = ConvLayer(16, 32)
mods = list(tst.children())

In [ ]:
mods

[Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU()]

In [ ]:
test_eq(len(mods), 3)
test_eq(mods[1].weight, torch.ones(32))
test_eq(mods[0].padding, (1,1))

In [ ]:
x = torch.randn(64, 16, 8, 8)#.cuda()

In [ ]:
x.shape

torch.Size([64, 16, 8, 8])

In [ ]:
tst = ConvLayer(1, 32, ndim=1, padding=0)

In [ ]:
mods = list(tst.children()); mods

[Conv1d(1, 32, kernel_size=(3,), stride=(1,), bias=False),
 BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU()]

In [ ]:
x = torch.randn(64, 1, 1700)#.cuda()

In [ ]:
tst(x)[0].shape

torch.Size([32, 1698])

In [ ]:
def get_nb_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum([np.prod(p.size()) for p in model_parameters])

In [ ]:
tst = xresnet18(ndim=1, c_in=1, ks=3, n_out=1)

In [ ]:
tst = xresnet101(ndim=1, c_in=1, ks=3, n_out=1)

In [ ]:
get_nb_params(tst) / 1e6

28.272737

In [ ]:
tst.parameters

<bound method Module.parameters of XResNet(
  (0): ConvLayer(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): ConvLayer(
    (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (2): ConvLayer(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (3): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): ResBlock(
      (convpath): Sequential(
        (0): ConvLayer(
          (0): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [ ]:
x = torch.randn(32, 1, 1700)
y = tst(x)

In [ ]:
y

TensorBase([[0.6363],
            [0.5567],
            [0.5903],
            [0.6276],
            [0.4324],
            [0.3994],
            [0.5164],
            [0.6200],
            [0.5415],
            [0.6101],
            [0.4702],
            [0.4930],
            [0.5256],
            [0.4059],
            [0.4147],
            [0.3187],
            [0.4753],
            [0.4550],
            [0.5613],
            [0.4620],
            [0.5145],
            [0.4408],
            [0.5057],
            [0.6801],
            [0.3946],
            [0.5191],
            [0.6256],
            [0.0948],
            [0.3550],
            [0.4861],
            [0.5112],
            [0.5925]], grad_fn=<AliasBackward0>)

## Resnet

In [ ]:
path = Path('../_data/kssl-mirs')
fnames = path.ls(); fnames

(#44101) [Path('../_data/kssl-mirs/180338'),Path('../_data/kssl-mirs/172221'),Path('../_data/kssl-mirs/177753'),Path('../_data/kssl-mirs/184798'),Path('../_data/kssl-mirs/53759'),Path('../_data/kssl-mirs/74947'),Path('../_data/kssl-mirs/176681'),Path('../_data/kssl-mirs/1855'),Path('../_data/kssl-mirs/175004'),Path('../_data/kssl-mirs/34499')...]

In [ ]:
get_spectra_files(fnames[0])

[Path('../_data/kssl-mirs/180338/260043XS04.csv'),
 Path('../_data/kssl-mirs/180338/260043XS01.csv'),
 Path('../_data/kssl-mirs/180338/260043XS02.csv'),
 Path('../_data/kssl-mirs/180338/260043XS03.csv')]

In [ ]:
df = get_target(fnames[0]); df

array([ 2.8077302, 22.033033 , 13.7791402])

In [ ]:
dblock = DataBlock(
    blocks=(SpectraBlock, RegressionBlock),
    get_x=get_spectra_files,
    get_y=partial(get_target, analytes=[725]),
    splitter=RandomSplitter(valid_pct=0.1, seed=42),
    item_tfms=[Transform(random_weighted_avg)]
)

In [ ]:
dsets = dblock.datasets(fnames)
dsets.train[0]

(tensor([[0.2258, 0.2260, 0.2263,  ..., 1.5891, 1.5789, 1.5701],
         [0.2639, 0.2641, 0.2644,  ..., 1.6350, 1.6246, 1.6143],
         [0.2816, 0.2819, 0.2821,  ..., 1.5537, 1.5427, 1.5320],
         [0.3072, 0.3074, 0.3076,  ..., 1.6325, 1.6217, 1.6133]]),
 tensor([0.2244]))

In [ ]:
dls = dblock.dataloaders(fnames, bs=16)

In [ ]:
dls.train.one_batch()

(tensor([[[0.2538, 0.2541, 0.2545,  ..., 1.5133, 1.5094, 1.5057]],
 
         [[0.4877, 0.4883, 0.4889,  ..., 1.6518, 1.6391, 1.6265]],
 
         [[0.1398, 0.1403, 0.1408,  ..., 1.3812, 1.3666, 1.3552]],
 
         ...,
 
         [[0.1906, 0.1908, 0.1909,  ..., 1.4994, 1.5049, 1.5104]],
 
         [[0.3844, 0.3844, 0.3845,  ..., 1.5356, 1.5320, 1.5228]],
 
         [[0.0872, 0.0876, 0.0880,  ..., 1.4735, 1.4687, 1.4652]]]),
 tensor([[0.2495],
         [0.4573],
         [0.5403],
         [1.4658],
         [0.0000],
         [0.0904],
         [0.2172],
         [0.0000],
         [0.0224],
         [0.3851],
         [2.0468],
         [0.2042],
         [0.5193],
         [0.1786],
         [0.5414],
         [0.2484]]))

In [ ]:
dls.valid.one_batch()

(tensor([[[0.3382, 0.3389, 0.3395,  ..., 1.6828, 1.6655, 1.6483]],
 
         [[1.0204, 1.0201, 1.0198,  ..., 1.3171, 1.3157, 1.3139]],
 
         [[0.2609, 0.2611, 0.2613,  ..., 1.5149, 1.5141, 1.5127]],
 
         ...,
 
         [[0.2286, 0.2288, 0.2290,  ..., 1.6269, 1.6229, 1.6186]],
 
         [[0.1504, 0.1508, 0.1512,  ..., 1.3450, 1.3361, 1.3288]],
 
         [[0.1300, 0.1301, 0.1301,  ..., 1.3425, 1.3299, 1.3182]]]),
 tensor([[0.4968],
         [0.3443],
         [0.0000],
         [0.0000],
         [0.9650],
         [0.6087],
         [1.3657],
         [0.0461],
         [1.0763],
         [1.4860],
         [0.7453],
         [0.0448],
         [2.7428],
         [0.1603],
         [0.6544],
         [0.5604]]))